In [12]:
import tiktoken 

In [4]:
with open("datask.txt", "r", encoding="utf-8") as f:
    text = f.read()

length = len(text)
print(length)

print("first one thousand charater")
chars = text[:1000]
print(chars)

1115394
first one thousand charater
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for

In [5]:
#all possible character
chars = set(text)
lst = sorted(list(chars))
print(lst)


vocabSize = len(lst)


['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
#Encoder and decoder
#we can use tiktoken too
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[x] for x in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello"))
print(decode([38, 1, 62, 62, 25]))


[48, 44, 22, 22, 60]
;n,,B


In [7]:
import torch
data = torch.Tensor(encode(lst[:1000]))
data

tensor([52., 47., 57., 35., 15.,  4., 62., 53.,  2., 19., 55., 38., 31., 11.,
        25., 28., 14., 51.,  0.,  9., 24., 40., 43.,  6.,  5.,  3., 23., 63.,
        45., 16., 42., 10., 30., 17., 26., 56., 59., 54., 13., 36., 29., 61.,
        27., 44., 39., 34., 48., 37., 12., 41., 22.,  7.,  1., 60., 21., 20.,
        32., 18., 33., 50., 64., 58.,  8., 49., 46.])

In [8]:
#lets make the train and test data split set kinda unsupervised learning
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

block_size = 8 #chunk of text to train the data on
train_data[:block_size+1]


tensor([52., 47., 57., 35., 15.,  4., 62., 53.,  2.])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")


when input is tensor([52.]), the target is 47.0
when input is tensor([52., 47.]), the target is 57.0
when input is tensor([52., 47., 57.]), the target is 35.0
when input is tensor([52., 47., 57., 35.]), the target is 15.0
when input is tensor([52., 47., 57., 35., 15.]), the target is 4.0
when input is tensor([52., 47., 57., 35., 15.,  4.]), the target is 62.0
when input is tensor([52., 47., 57., 35., 15.,  4., 62.]), the target is 53.0
when input is tensor([52., 47., 57., 35., 15.,  4., 62., 53.]), the target is 2.0


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.long(), y.long()

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

print(xb.dtype)


inputs:
torch.Size([4, 8])
tensor([[28, 14, 51,  0,  9, 24, 40, 43],
        [53,  2, 19, 55, 38, 31, 11, 25],
        [27, 44, 39, 34, 48, 37, 12, 41],
        [52, 47, 57, 35, 15,  4, 62, 53]])
targets:
torch.Size([4, 8])
tensor([[14, 51,  0,  9, 24, 40, 43,  6],
        [ 2, 19, 55, 38, 31, 11, 25, 28],
        [44, 39, 34, 48, 37, 12, 41, 22],
        [47, 57, 35, 15,  4, 62, 53,  2]])
----
when input is [28] the target: 14
when input is [28, 14] the target: 51
when input is [28, 14, 51] the target: 0
when input is [28, 14, 51, 0] the target: 9
when input is [28, 14, 51, 0, 9] the target: 24
when input is [28, 14, 51, 0, 9, 24] the target: 40
when input is [28, 14, 51, 0, 9, 24, 40] the target: 43
when input is [28, 14, 51, 0, 9, 24, 40, 43] the target: 6
when input is [53] the target: 2
when input is [53, 2] the target: 19
when input is [53, 2, 19] the target: 55
when input is [53, 2, 19, 55] the target: 38
when input is [53, 2, 19, 55, 38] the target: 31
when input is [53, 2, 19,

In [13]:


import torch
import torch.nn as nn
from torch.nn import functional as f 

class BigramlanguageModel(nn.Module):
    def __init__(self, vocabSize):
        super().__init__()
        self.tokenEmbedding_table = nn.Embedding(vocabSize, vocabSize)
    
    def forward(self, idx, targets):
        logits = self.tokenEmbedding_table(idx) #BTC

        if target == None:
            loss = None

        else:    

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = f.softmax(logits, dim=-1)
            idx_next = torch.multinominal(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
            return idx


m = BigramlanguageModel(vocabSize=vocabSize)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.9254, grad_fn=<NllLossBackward0>)
